# Process and barcode sequencing reads
This script takes paired read (read 1: barcode + UMI (27bp), read 2: staggers + UMI + partial barcode (49bp)) fastq files and does the following:
1. trims read 2 adapter sequences to recover barcode+UMI sequence
2. merges read 1 and 2 with FLASH
3. identifies and counts barcodes

In [1]:
import glob
import os
import subprocess
import regex
import gzip
from Bio import SeqIO
import pandas as pd
import multiprocessing as mp
import numpy as np

# check number of available cores
len(os.sched_getaffinity(0))

20

## Inspect fastq files for quality control using FastQC

In [2]:
# !mkdir ~/scratch/yeast/crispey3/gxg_bya_aug2021/fastq/fastqc/
# !fastqc -q -o ~/scratch/yeast/crispey3/gxg_bya_aug2021/fastq/fastqc/ ~/scratch/yeast/crispey3/gxg_bya_aug2021/fastq/*fastq.gz
!mkdir ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/fastqc/
!fastqc -q -o ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/fastqc/ ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/18146FL-40*fastq.gz

## Summarize FastQC output with MultiQC

In [3]:
# !multiqc -o ~/scratch/yeast/crispey3/gxg_bya_aug2021/fastq/fastqc/ ~/scratch/yeast/crispey3/gxg_bya_aug2021/fastq/fastqc/
!multiqc -o ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/fastqc/ ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/fastqc/

[WARNING]         multiqc : MultiQC Version v1.11 now available!
[INFO   ]         multiqc : This is MultiQC v1.9
[INFO   ]         multiqc : Template    : default
[INFO   ]         multiqc : Searching   : /home/users/rang/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/fastqc
Searching 300 files..  [####################################]  100%          
[INFO   ]          fastqc : Found 150 reports
[ERROR  ]        bargraph : ############### Error making MatPlotLib figure! Falling back to HighCharts.
[ERROR  ]       linegraph : ############### Error making MatPlotLib figure! Falling back to HighCharts.
[ERROR  ]       linegraph : ############### Error making MatPlotLib figure! Falling back to HighCharts.
[ERROR  ]       linegraph : ############### Error making MatPlotLib figure! Falling back to HighCharts.
[ERROR  ]       linegraph : ############### Error making MatPlotLib figure! Falling back to HighCharts.
[ERROR  ]       linegraph : ############### Error making MatPlotLib figure

## Map fastq file names to sample names

In [4]:
# key to map fastq names to output names
seqID_to_sampleName = {}
# sample_key_file = "/home/users/rang/scratch/yeast/crispey3/gxg_bya_aug2021/SampleKey-18146-35.txt"
sample_key_file = "/home/users/rang/scratch/yeast/crispey3/gxg_5comp_aug2021/SampleKey-18146-40.txt"
with open(sample_key_file, 'r') as sample_key:
    sample_key.readline() # skip header
    for line in sample_key:
        seqID, sampleName = line.rstrip().split("\t")
        sampleName = sampleName.replace("_","-")
        seqID_to_sampleName[seqID] = sampleName


## Trim read 2 adapters with cutadapt
Remove staggers, leaving the UMI+partial barcode sequence (19-26bp, may be shorter depending on quality trimming)

In [5]:
# working directory with fastq files
# working_dir="/home/users/rang/scratch/yeast/crispey3/gxg_bya_aug2021/fastq/"
working_dir="/home/users/rang/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/"
os.chdir(working_dir)

# get read 2 files for trimming
# fastq_list = sorted([os.path.abspath(x) for x in glob.glob("*R2_001.fastq.gz")])
fastq_list = sorted([os.path.abspath(x) for x in glob.glob("./run*/18146FL-40*R2_001.fastq.gz")])

In [6]:
# cutadapt parameters to trim read 2 to get barcode+UMI (27bp)
adapter_5prime = 'GGCCAGTTTAAACTT'
adapter_3prime = 'GCATGGC'

num_of_cores = len(os.sched_getaffinity(0))
err = 0.2 # fraction tolerated for adapter matching
min_r2_length = 12 # R2 must contain at least UMI sequence and some of SphI linker
output_dir_name = 'trimmed'

In [7]:
# store sample key in regex pattern
pattern = regex.compile('|'.join(seqID_to_sampleName.keys()))

# trim read 2, filter untrimmed read pairs
for fastq_path in fastq_list:
    fastq_dir = os.path.dirname(fastq_path)
    output_dir = fastq_dir + "/"+output_dir_name+"/"
    os.makedirs(output_dir, exist_ok=True)
    
    # rename output files by sample key stored in seqID_to_sampleName 
    fastq_file = os.path.basename(fastq_path)
    output_file_r2 = pattern.sub(lambda x: seqID_to_sampleName[x.group()], fastq_file).replace("_001.fastq.gz", "_001_trimmed.fastq.gz")
    output_file_r1 = output_file_r2.replace("_R2_", "_R1_")

#     print('Trimming: ' + fastq_path)
    
    cutadapt_cmd = ["cutadapt", "-g", adapter_5prime+"..."+adapter_3prime, #adapter_5prime, 
                    "-j", str(num_of_cores), 
                    "-e", str(err), 
                    #"-q", "20", # use -q for miseq/hiseq quality trimming
                    "--nextseq-trim", "20", # use this option for nextseq quality trimming
                    "--discard-untrimmed", "-m", str(min_r2_length), 
                    "--pair-filter=first", 
                    "-o", output_dir+output_file_r2, "-p", output_dir+output_file_r1,
                    fastq_path, fastq_path.replace("L001_R2_", "L001_R1_")]
    
    subprocess.run(cutadapt_cmd)

#     print("Output files:")
#     print(output_file_r2)
#     print(output_file_r1)
#     print()
    
print('Done trimming!')

Trimming: /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/18146FL-40-01-01_S1_L001_R2_001.fastq.gz
Output files:
BYA-t1-1_S1_L001_R2_001_trimmed.fastq.gz
BYA-t1-1_S1_L001_R1_001_trimmed.fastq.gz

Trimming: /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/18146FL-40-01-02_S2_L001_R2_001.fastq.gz
Output files:
BYA-t1-2_S2_L001_R2_001_trimmed.fastq.gz
BYA-t1-2_S2_L001_R1_001_trimmed.fastq.gz

Trimming: /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/18146FL-40-01-03_S3_L001_R2_001.fastq.gz
Output files:
BYA-t1-3_S3_L001_R2_001_trimmed.fastq.gz
BYA-t1-3_S3_L001_R1_001_trimmed.fastq.gz

Trimming: /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/18146FL-40-01-04_S4_L001_R2_001.fastq.gz
Output files:
BYA-t4-1_S4_L001_R2_001_trimmed.fastq.gz
BYA-t4-1_S4_L001_R1_001_trimmed.fastq.gz

Trimming: /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/18146FL-40-01-05_S5_L001_R2_001.fastq.gz
Output files:
BYA-t4-2_S5_L001_R2_001

Output files:
RM-t7-3_S39_L001_R2_001_trimmed.fastq.gz
RM-t7-3_S39_L001_R1_001_trimmed.fastq.gz

Trimming: /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/18146FL-40-01-40_S40_L001_R2_001.fastq.gz
Output files:
RM-t10-1_S40_L001_R2_001_trimmed.fastq.gz
RM-t10-1_S40_L001_R1_001_trimmed.fastq.gz

Trimming: /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/18146FL-40-01-41_S41_L001_R2_001.fastq.gz
Output files:
RM-t10-2_S41_L001_R2_001_trimmed.fastq.gz
RM-t10-2_S41_L001_R1_001_trimmed.fastq.gz

Trimming: /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/18146FL-40-01-42_S42_L001_R2_001.fastq.gz
Output files:
RM-t10-3_S42_L001_R2_001_trimmed.fastq.gz
RM-t10-3_S42_L001_R1_001_trimmed.fastq.gz

Trimming: /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/18146FL-40-01-43_S43_L001_R2_001.fastq.gz
Output files:
RM-t13-1_S43_L001_R2_001_trimmed.fastq.gz
RM-t13-1_S43_L001_R1_001_trimmed.fastq.gz

Trimming: /scratch/users/rang/yeast/crispey

## Merge read 1 and read 2 with FLASH to produce final barcode+UMI sequence

In [8]:
# working_dir="/home/users/rang/scratch/yeast/crispey3/gxg_bya_aug2021/fastq/trimmed/"
working_dir="/home/users/rang/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/"
os.chdir(working_dir)

# fastq_list = sorted([os.path.abspath(x) for x in glob.glob("*R1_001_trimmed.fastq.gz")])
fastq_list = sorted([os.path.abspath(x) for x in glob.glob("./run*/trimmed/*R1_001_trimmed.fastq.gz")])

In [9]:
# FLASH parameters
min_overlap = 8 # cannot be longer than the shorter read.
max_overlap = 12 # cannot be longer than the shorter read.
max_mismatch = 0.25 #default is 0.25, set 0.4 to tolerate lower seq quality
num_of_cores = len(os.sched_getaffinity(0))
output_dir_name = 'merged'

In [10]:
# use FLASH to merge trimmed-filtered read 2 and read 1 data to produce final 27bp sequence containing barcode and UMI data
for fastq_path in fastq_list:
    fastq_dir = os.path.dirname(fastq_path)
    output_dir = fastq_dir + "/"+output_dir_name+"/"
    os.makedirs(output_dir, exist_ok=True)
    
    # check output file naming 
    output_prefix = os.path.basename(fastq_path).split("_")[0]+"_barcode"
#     print('Merging', fastq_path, 'and', fastq_path.replace("_R1_", "_R2_"))
    
    flash_cmd = ["flash", "-m", str(min_overlap), "-M", str(max_overlap),
                 "-x", str(max_mismatch), #"-O", # use -O if innie-only merging does not work
                 "-t", str(num_of_cores),
                 "-o", output_prefix, "-d", output_dir, 
                 "--compress", 
                 fastq_path, fastq_path.replace("_R1_", "_R2_")]
    subprocess.run(flash_cmd)
#     print(output_prefix, "merged")

print('Done merging!')

Merging /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/BYA-t1-1_S1_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/BYA-t1-1_S1_L001_R2_001_trimmed.fastq.gz
BYA-t1-1_barcode merged
Merging /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/BYA-t1-2_S2_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/BYA-t1-2_S2_L001_R2_001_trimmed.fastq.gz
BYA-t1-2_barcode merged
Merging /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/BYA-t1-3_S3_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/BYA-t1-3_S3_L001_R2_001_trimmed.fastq.gz
BYA-t1-3_barcode merged
Merging /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/BYA-t10-1_S10_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/BYA-t10-1_S10_L

RM-t1-2_barcode merged
Merging /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/RM-t1-3_S33_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/RM-t1-3_S33_L001_R2_001_trimmed.fastq.gz
RM-t1-3_barcode merged
Merging /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/RM-t10-1_S40_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/RM-t10-1_S40_L001_R2_001_trimmed.fastq.gz
RM-t10-1_barcode merged
Merging /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/RM-t10-2_S41_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/RM-t10-2_S41_L001_R2_001_trimmed.fastq.gz
RM-t10-2_barcode merged
Merging /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/RM-t10-3_S42_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run

YPS-t10-1_barcode merged
Merging /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/YPS-t10-2_S71_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/YPS-t10-2_S71_L001_R2_001_trimmed.fastq.gz
YPS-t10-2_barcode merged
Merging /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/YPS-t10-3_S72_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/YPS-t10-3_S72_L001_R2_001_trimmed.fastq.gz
YPS-t10-3_barcode merged
Merging /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/YPS-t13-1_S73_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/YPS-t13-1_S73_L001_R2_001_trimmed.fastq.gz
YPS-t13-1_barcode merged
Merging /scratch/users/rang/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/YPS-t13-2_S74_L001_R1_001_trimmed.fastq.gz and /scratch/users/rang/yeast/crispey3/gxg_5comp_au

## (optional) Join processed fastq files across multiple sequencing runs for each sample

In [ ]:
# note cannot run bash for loop within jupyter notebook
# !mkdir ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/final/
# !for file in ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run1/trimmed/merged/*.extendedFrags.fastq.gz; do
# !cat ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/run*/trimmed/merged/$(basename "$file") > ~/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/final/$(basename "$file")
# !done

## Count barcodes
Counting barcodes consists of several steps. First, parse each fastq file and count all sequences. After assembling into an initial sequences counts matrix, extract the barcode and UMI sequences and map them to a reference table of barcodes and UMIs. Counts for ID-able sequences are onsolidated into a final counts matrix for input to DESeq2.

It is possible to parallelize this process if handling multiple samples across different conditions. However, it is more efficient to put all counts across all conditions into a single matrix and run the barcode mapping a single time, since the barcodes are the same across all conditions.

In [11]:
def count_seqs(fastq_file, min_seq_length, max_seq_length):
    '''
    Parses a fastq file and counts sequences. Returns dict of counts
    '''
    seq_counts_dict = {}
    # parse fastq
    with gzip.open(fastq_file, 'rt') as fastq:
        for read in SeqIO.parse(fastq, "fastq"):
            # filter for sequences within min/max length and contains no N's
            if min_seq_length <= len(read.seq) <= max_seq_length and read.seq.count("N")==0:
                sequence = str(read.seq)
                # count sequence
                try:
                    seq_counts_dict[sequence] += 1
                except KeyError:
                    seq_counts_dict[sequence] = 1
    
    return seq_counts_dict

    
def map_seq_to_barcode_umi(seq, barcode_table, umi_list, barcode_length, umi_length, linker_seq):
    '''
    splits a sequence into barcode and UMI, maps to barcode table and UMI list to assign ID
    does NOT do UMI mapping if umi_list is empty.
    '''
    final_id = 'UNKNOWN'
    
    barcode, umi = split_barcode_umi_from_seq(seq, barcode_length, umi_length, linker_seq)
    
    if len(barcode)<barcode_length/2 or len(umi)<umi_length/2:
        # Barcode/UMI too short
        return final_id

    # assign barcode ID
    barcode_id = assign_barcode(barcode=barcode, barcode_table=barcode_table, error=1)
    if barcode_id == 'UNKNOWN':
        # Barcode cannot be identified
        return barcode_id
    else:
        # add barcode ID to final ID
        final_id = barcode_id

    # assign UMI ID (if applicable)
    if len(umi_list)>0:
        umi_id = assign_umi(umi=umi, umi_list=umi_list, error=1)
        if umi_id == 'UNKNOWN':
            # UMI cannot be identified
            return umi_id
        else:
            # add UMI ID to final ID
            final_id = '-'.join([final_id, str(umi_id)])
    
    return final_id
    

def split_barcode_umi_from_seq(seq, barcode_length, umi_length, linker_seq):
    '''
    Splits seq by linker_seq and returns barcode and UMI sequence
    If linker seq cannot be found (e.g. sequencing error) or yields multiple splits,
    fall back to splitting by base position.
    '''
    barcode = ''
    umi = ''
    
    try:
        # split by linker
        barcode, umi = seq.split(linker_seq) # can try error tolerant regex?
    except ValueError:
        # split by base position
        if barcode_length>0 and umi_length==0:
            barcode = seq[:barcode_length]
        else:
            umi = seq[-umi_length:]
            barcode = seq[:-(umi_length+len(linker_seq))] # may return partial barcodes for short sequences

    return (barcode, umi)


def assign_barcode(barcode, barcode_table, error):
    '''
    Searches barcode table for barcode sequence and returns unique barcode ID
    Tries perfect match first, then error-tolerant regex
    '''
    if barcode == '':
        barcode_id = 'UNKNOWN'
    else:
        try:
            # search for perfect match
            barcode_id = barcode_table.loc[barcode, 'Unique_ID']
        except KeyError:
            # search by error-tolerant regex
            pattern = "(?:"+barcode+"){s<="+str(error)+"}"
            search = [bool(regex.search(pattern, x)) for x in barcode_table.index]
            if sum(search)==1:
                barcode_id = barcode_table.loc[search, 'Unique_ID'][0]
            else:
                # barcode cannot be identified
                barcode_id = 'UNKNOWN'
    
    return barcode_id


def assign_umi(umi, umi_list, error):
    '''
    Searches umi list for umi sequence and returns 1-index position as umi ID
    Tries perfect match first, then error-tolerant regex
    '''
    if umi == '':
        umi_id = 'UNKNOWN'
    else:
        try:
            # search for perfect match
            umi_id = umi_list.index(umi)+1
        except ValueError:
            # search by error-tolerant regex
            pattern = "(?:"+umi+"){s<="+str(error)+"}"
            search = [bool(regex.search(pattern, x)) for x in umi_list]
            if sum(search) == 1:
                umi_id = search.index(True)+1
            else:
                # UMI cannot be identified
                umi_id = 'UNKNOWN'
    
    return umi_id


In [12]:
# working_dir="/home/users/rang/scratch/yeast/crispey3/gxg_bya_aug2021/fastq/trimmed/merged/"
working_dir="/home/users/rang/scratch/yeast/crispey3/gxg_5comp_aug2021/fastq/final/"
os.chdir(working_dir)

#output directory
# output_dir = "/home/users/rang/scratch/yeast/crispey3/pool1_scm_aug2021/counts/"
# output_dir = "/home/users/rang/scratch/yeast/crispey3/gxg_bya_aug2021/counts/"
output_dir = "/home/users/rang/scratch/yeast/crispey3/gxg_5comp_aug2021/counts/"

# merged reads to count barcodes from
fastq_list = sorted(glob.glob("*extendedFrags.fastq.gz")) # check for file name
# sample names for each fastq
sample_name_list = [fastq_file.split("_")[0] for fastq_file in fastq_list] # adjust accordingly to generate sample name for counts matrix

# sequence counts file (before combining)
seq_counts_filename = "seq_counts.txt"

# mapped barcode-UMI counts file
barcode_counts_filename = "barcode_counts.txt"


# open barcode reference file
barcode_reference_file = '/home/users/rang/crispey3/library_design/Input/12BP_PBCs_well_grouped.csv'
barcode_table = pd.read_csv(barcode_reference_file, index_col=1)

# approved list of UMIs used in cloning CRISPEY3 plasmid
umi_list = ['ACGCGTGAA',
            'ATGTGGCTC',
            'CAGAGGATC',
            'CTGTGGCAA',
            'GTGTGATTC',
            'TAGAGGACT',] # only first 6 UMIs were included in cloning CRISPEY3 libaries
#             'AAGAGCCTC',
#             'AAGAGGAGG',
#             'ATGTGCGAA',
#             'ATGTGTAGG',
#             'CAGAGCCAA',
#             'CTGTGATGG',
#             'CTGTGTATC',
#             'GAGAGGAAA',
#             'TCGCGGTAA',
#             'TTGTGCGTC']
umi_list = sorted(umi_list)


In [13]:
# count sequences in each fastq file
fastq_dict = dict(zip(sample_name_list, fastq_list))
with mp.Pool(min(len(os.sched_getaffinity(0)), len(fastq_list))) as pool:
    seq_counts_df = {sample_name : pool.apply_async(count_seqs, (fastq_file, 20, 27)) for sample_name, fastq_file in fastq_dict.items()}
    seq_counts_df = {sample_name : res.get() for sample_name, res in seq_counts_df.items()}
    
# merge to dataframe
seq_counts_df = pd.DataFrame.from_dict(seq_counts_df, orient="columns")
seq_counts_df.index.name = 'sequence'

# write to file to inspect
os.makedirs(output_dir, exist_ok=True)
seq_counts_df.to_csv(output_dir+seq_counts_filename, sep="\t")


In [13]:
# seq_counts_df = pd.read_csv(output_dir+seq_counts_filename, sep="\t", index_col='sequence')

In [14]:
# map each sequence in seq_counts_df to barcode-UMI ID
mapped_counts_df = seq_counts_df.copy()
mapped_counts_df = mapped_counts_df.reset_index()

# filter out low count barcodes (computationally expensive to map these rare barcodes, minimal impact to total count (~2%))
cutoff = 3
mapped_counts_df = mapped_counts_df.loc[mapped_counts_df.sum(axis=1)>=cutoff]

# do multiprocessing for sequence mapping
num_of_cores = len(os.sched_getaffinity(0))
with mp.Pool(num_of_cores) as pool:
    barcode_umi_id_list = [pool.apply_async(map_seq_to_barcode_umi, (seq, barcode_table, umi_list, 12, 9, 'GCATGC')) for seq in mapped_counts_df['sequence']]
    barcode_umi_id_list = [res.get() for res in barcode_umi_id_list]
    
# mapped_counts_df['barcode_umi_id'] = [x for res in barcode_umi_id_lists for x in res] #pd.concat(barcode_umi_id_lists)
mapped_counts_df['barcode_umi_id'] = barcode_umi_id_list
display(mapped_counts_df)

# remove unknowns
mapped_counts_df = mapped_counts_df.query('barcode_umi_id!="UNKNOWN"')

# consolidate counts
mapped_counts_df = mapped_counts_df.groupby('barcode_umi_id').sum().fillna(0).astype(int)
mapped_counts_df.index.name = 'barcode'
display(mapped_counts_df)

# write all counts to output file
os.makedirs(output_dir, exist_ok=True)
mapped_counts_df.to_csv(output_dir+barcode_counts_filename, sep="\t")


# # one-liner to map sequences to barcode-umi IDs
# # warning: single-threaded, expected to be slow
# mapped_counts_df['barcode_umi_id'] = mapped_counts_df['sequence'].apply(map_seq_to_barcode_umi, args=(barcode_table, umi_list, 12, 9, 'GCATGC'))


,sequence,BYA-t1-1,BYA-t1-2,BYA-t1-3,BYA-t10-1,BYA-t10-2,BYA-t10-3,BYA-t13-1,BYA-t13-2,BYA-t13-3,...,YPS-t13-1,YPS-t13-2,YPS-t13-3,YPS-t4-1,YPS-t4-2,YPS-t4-3,YPS-t7-1,YPS-t7-2,YPS-t7-3,barcode_umi_id
0,ATTCACTTAAGCGCATGCCAGAGGATC,304.0,291.0,292.0,212.0,235.0,232.0,138.0,200.0,145.0,...,115.0,90.0,106.0,60.0,77.0,88.0,96.0,87.0,54.0,053_046-3
1,CACCGTCTCCGTGCATGCATGTGGCTC,525.0,492.0,510.0,643.0,563.0,512.0,644.0,528.0,432.0,...,469.0,592.0,597.0,509.0,489.0,509.0,475.0,518.0,289.0,077_044-2
2,ATTCGAGTGCTCGCATGCATGTGGCTC,1698.0,1679.0,1746.0,1691.0,1648.0,1560.0,1507.0,1469.0,1371.0,...,861.0,675.0,710.0,886.0,816.0,778.0,880.0,832.0,471.0,037_113-2
3,CAAAGCGCTCACGCATGCATGTGGCTC,4194.0,4198.0,4209.0,3918.0,4326.0,3811.0,3341.0,3898.0,3611.0,...,3503.0,3589.0,3263.0,3566.0,3552.0,3413.0,3464.0,3676.0,2174.0,053_101-2
4,ATACTCAACAACGCATGCCTGTGGCAA,386.0,379.0,377.0,374.0,330.0,402.0,313.0,274.0,440.0,...,386.0,419.0,417.0,341.0,387.0,389.0,303.0,432.0,224.0,057_118-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292468,GCTTTGTGGATTGCATGCCTGCGGCAA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,066_026-4
3292882,GTGAAGAGTACAGCATGCATGTGGATC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,045_050-2
3308286,TACCGAGAACACGCATGCATGTGGCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,069_058-2
3317085,CGTAGATTGTTTGCATGCATGTGGCTC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,049_014-2


,BYA-t1-1,BYA-t1-2,BYA-t1-3,BYA-t10-1,BYA-t10-2,BYA-t10-3,BYA-t13-1,BYA-t13-2,BYA-t13-3,BYA-t4-1,...,YPS-t10-3,YPS-t13-1,YPS-t13-2,YPS-t13-3,YPS-t4-1,YPS-t4-2,YPS-t4-3,YPS-t7-1,YPS-t7-2,YPS-t7-3
barcode,,,,,,,,,,,,,,,,,,,,,
001_001-6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001_003-3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001_003-4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001_004-3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
001_010-4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ladder_043-2,0,2,4,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
Ladder_043-3,2,4,5,0,2,2,1,1,0,0,...,0,0,0,0,0,0,1,0,0,0
Ladder_043-4,2,1,0,0,0,2,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0


## (optional) Combine counts across UMIs per barcode
The counts of different UMIs of the same barcode can be added together to produce a stacked counts matrix

In [15]:
# combine counts from different UMIs of the same barcode
stacked_counts_filename = "stacked_barcode_counts.txt"

stacked_counts_df = mapped_counts_df.groupby(by=lambda x: x.split('-')[0]).sum()
stacked_counts_df.index.name = 'barcode'
stacked_counts_df.to_csv(output_dir+stacked_counts_filename, sep="\t")

In [2]:
# # adapt ladder fastq files by trimming to new seq format (R1: 27bp, R2: 49bp)
# r1_len = 27
# r2_len = 49

# # working directory with fastq files
# working_dir="/home/users/rang/scratch/yeast/tmp/ladder_pilot_adapted/fastq/"
# os.chdir(working_dir)

# # trim R1
# fastq_list = sorted([os.path.abspath(x) for x in glob.glob("*R1_001.fastq.gz")])
# for fastq_path in fastq_list:
#     output_path = fastq_path.replace("_R1_001", "_R1_adapted")
#     cutadapt_cmd = ["cutadapt", "-l", str(r1_len),
#                     "-o", output_path, fastq_path]
#     subprocess.run(cutadapt_cmd)
    
# # trim R2
# fastq_list = sorted([os.path.abspath(x) for x in glob.glob("*R2_001.fastq.gz")])
# for fastq_path in fastq_list:
#     output_path = fastq_path.replace("_R2_001", "_R2_adapted")
#     cutadapt_cmd = ["cutadapt", "-l", str(r2_len),
#                     "-o", output_path, fastq_path]
#     subprocess.run(cutadapt_cmd)

In [ ]:
# # one-step count_barcodes function
# # warning: less efficient since barcode-UMI mapping is done per sample, rather than a single time after all sequences are counted

# def count_barcodes(fastq_file, barcode_table, umi_list, 
#                    min_seq_length=20, barcode_length=12, umi_length=9, linker_seq='GCATGC'):
#     '''
#     parses fastq file to count sequences, then extracts barcode-UMI info from sequences and assigns ID based
#     on provided reference barcode_table and umi_list. Finally, consolidates counts by assigned ID and returns
#     a dict of barcode counts.
#     does NOT do UMI mapping if umi_list is empty.
#     '''
#     # set max_seq_length
#     max_seq_length = barcode_length+umi_length+len(linker_seq) # this could be adjusted to allow insertions
#     # alphabetical sort umi_list
#     umi_list = sorted(umi_list)
    
#     # count raw sequences
#     seq_counts = count_seqs(fastq_file, min_seq_length, max_seq_length)
    
#     # consolidate barcode counts
#     barcode_counts_dict = {}
#     for seq, count in seq_counts.items():
#         # assign barcode-UMI ID
#         assigned_id = map_seq_to_barcode_umi(seq, barcode_table, umi_list, barcode_length, umi_length, linker_seq)
#         if assigned_id:
#             try:
#                 barcode_counts_dict[assigned_id] += count
#             except KeyError:
#                 barcode_counts_dict[assigned_id] = count
    
#     return barcode_counts_dict


# fastq_dict = dict(zip(sample_name_list, fastq_list))
# with mp.Pool(min(len(os.sched_getaffinity(0)), len(fastq_list))) as pool:
#     all_counts_df = {sample_name : pool.apply_async(count_barcodes, (fastq_file, barcode_table, umi_list)) for sample_name, fastq_file in fastq_dict.items()}
#     all_counts_df = {sample_name : res.get() for sample_name, res in all_counts_df.items()}
    
# # write all counts to output file
# os.makedirs(output_dir, exist_ok=True)
# all_counts_df = pd.DataFrame.from_dict(all_counts_df, orient="columns")
# all_counts_df.index.name = 'barcode'
# all_counts_df.to_csv(output_dir+counts_filename, sep="\t")